In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import logging
import json

In [3]:
data = pd.read_csv('../data/vacancies_ru.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
0,v_862116,Смотритель музейный,c_162972,"['Пользователь ПК', 'Работа в команде', 'Умени...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5,5,1
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"['Активные продажи', 'Холодные продажи', 'Кред...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5,5,1
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,NaN,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер,81,5806,109013,5,5,3
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,NaN,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,Пекарь,52,4218,235992,5,5,1
4,v_312507,Торговый представитель (г. Абакан),c_206699,"['Продуктивность', 'Клиентоориентированность',...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209567,v_2484959,Сборщик-упаковщик,c_203256,"['Пользователь ПК', 'Умение работать в команде...",40000.0,90000.0,RUR,a_5387,ar_71,full,flexible,noExperience,Сборщик-упаковщик,74,4876,114733,5,2,1
2209568,v_205163,Сварщик на полуавтомат,c_158695,"['Желание работать и зарабатывать', 'Высокая э...",80000.0,130000.0,RUR,a_5527,ar_69,full,fullDay,between1And3,Сварщик на полуавтомат,71,5032,65219,5,5,2
2209569,v_639897,Главный инженер / Технический директор,c_209365,"['Контроль исправности оборудования', 'Инженер...",200000.0,NaN,RUR,a_1756,ar_41,full,fullDay,between1And3,Главный инженер,41,842,121520,5,5,2
2209570,v_1636531,"Провизор/Фармацевт (г.Адыгейск, 20 км от Красн...",c_246244,"['Предпечатная подготовка', 'Статистический ан...",25000.0,NaN,RUR,a_3403,ar_60,full,fullDay,noExperience,Фармацевт,62,2673,162497,5,5,1


In [4]:
url = "https://api.hh.ru/professional_roles"

response = requests.get(url)

if response.status_code == 200:
    data_list = response.json()

In [5]:
data_list['categories'][0]

{'id': '19',
 'name': 'Автомобильный бизнес',
 'roles': [{'id': '4',
   'name': 'Автомойщик',
   'accept_incomplete_resumes': True,
   'is_default': False,
   'select_deprecated': False,
   'search_deprecated': False},
  {'id': '5',
   'name': 'Автослесарь, автомеханик',
   'accept_incomplete_resumes': True,
   'is_default': False,
   'select_deprecated': False,
   'search_deprecated': False},
  {'id': '62',
   'name': 'Мастер-приемщик',
   'accept_incomplete_resumes': True,
   'is_default': False,
   'select_deprecated': False,
   'search_deprecated': False},
  {'id': '70',
   'name': 'Менеджер по продажам, менеджер по работе с клиентами',
   'accept_incomplete_resumes': False,
   'is_default': False,
   'select_deprecated': False,
   'search_deprecated': False}]}

In [84]:
filename = 'vacancy_hh_dataset_all.csv'
currency_code = 'RUR'
order_by = 'publication_time'
period = 30

url_hh = 'https://api.hh.ru/'
url_roles = url_hh + 'professional_roles'
url_vacancies = url_hh + 'vacancies' 

In [52]:
prof_roles_codes = []
for i in range(len(data_list['categories'])):
    for j in range(len(data_list['categories'][i]['roles'])):
        prof_roles_codes.append(int(data_list['categories'][i]['roles'][j]['id']))
            
prof_roles_codes = list(set(prof_roles_codes))

In [53]:
len(prof_roles_codes)

174

In [54]:
columns = ['vacancy_id', 'name', 'company_id', 'keySkills', 'compensation_from',
           'compensation_to', 'compensation_currencyCode', 'area_id', 'employment',
           'workSchedule', 'workExperience', 'description', 'published_at']

In [59]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)

In [60]:
logging.basicConfig(
    filename='hh_logs.log',
    level=logging.INFO,
    format = '%(asctime)s - %(levelname)s - %(message)s'
    )

In [198]:
def make_dataset(params, page_cnt):
    
    df = pd.DataFrame(columns=columns)
    ind = 0

    for code in prof_roles_codes:
        ind += 1
        vacancies = []
        progress_bar = tqdm(total=page_cnt, desc=f"Professional role {code}, {ind}/{len(prof_roles_codes)}")
        for i in range(page_cnt):
            progress_bar.update(1)
            
            params.update({'professional_role': code, 'page': i})

            try:
                response = requests.get(url_vacancies, params=params)
            except Exception:
                continue

            if response.status_code == 200:
                vac_page = response.json()

                for j in range(len(vac_page['items'])):

                    vac = vac_page['items'][j]
                    row = {column: None for column in columns}

                    row['vacancy_id'] = vac.get('id')

                    try:
                        response = requests.get(f"{url_vacancies}/{row['vacancy_id']}")
                    except Exception:
                        continue

                    if response.status_code == 200:
                        vac = response.json()

                        row['name'] = vac.get('name')
                        row['company_id'] = vac['employer'].get('id')

                        if vac.get('salary') is not None:
                            row['compensation_from'] = vac['salary'].get('from')
                            row['compensation_to'] = vac['salary'].get('to')
                            row['compensation_currencyCode'] = vac['salary'].get('currency')
                        else:
                            row['compensation_from'] = None
                            row['compensation_to'] = None
                            row['compensation_currencyCode'] = None

                        row['area_id'] = vac['area'].get('id')
                        row['employment'] = vac.get('employment').get('id') if vac.get('employment') else None
                        row['workSchedule'] = vac.get('schedule').get('id') if vac.get('schedule') else None
                        row['workExperience'] = vac.get('experience').get('id') if vac.get('experience') else None
                        row['description'] = BeautifulSoup(vac['description'], 'html.parser').get_text()
                        row['keySkills'] = [item['name'] for item in vac.get('key_skills', [])]
                        row['published_at'] = datetime.strptime(vac.get('published_at')[:19],
                                                                      '%Y-%m-%dT%H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')
                        row['alternate_url'] = vac.get('alternate_url')
                        row['prof_codes'] = vac.get('professional_roles')[0].get('id') if vac.get('professional_roles') else None

                        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
                        logging.info(f'Successful concatenation. Shape of dataframe: {df.shape}')

        progress_bar.close()
        print(df.shape)
    
    return df

In [17]:
# коды городов
with open('../data/mapping/names_to_region_id.json', 'r') as f:
    codes = json.load(f)

In [292]:
cities = ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород',
        'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 
        'Воронеж', 'Волгоград', 'Пермь']

In [293]:
area_codes = []

for city in cities:
    area_codes.append(codes[city])

print(area_codes)

['1', '2', '4', '3', '88', '66', '104', '54', '78', '99', '76', '68', '53', '26', '24', '72']


In [294]:
# апдейт кодов города для датасета
# area_codes.remove('1')
# print(area_codes)

['2', '4', '3', '88', '66', '104', '54', '78', '99', '76', '68', '53', '26', '24', '72']


In [109]:
page_cnt = 20
item_per_page = 50

In [297]:
df = pd.DataFrame(columns=columns)

for code in tqdm(area_codes, desc=f"Processing areas {code}", total=len(area_codes)):
    
    params = {
        'area': code,
        'currency_code': currency_code, 
        'per_page': item_per_page,
        'period': period,
        'order_by': order_by
    } 

    df_part = make_dataset(params, page_cnt)
    df = pd.concat([df, df_part], ignore_index=True)
    
    break

Processing areas 2:   0%|                                | 0/15 [00:00<?, ?it/s]

Professional role 1, 1/174:   0%|                        | 0/20 [00:00<?, ?it/s]

Professional role 1, 1/174:  10%|█▌              | 2/20 [00:06<01:01,  3.43s/it]

Professional role 1, 1/174:  15%|██▍             | 3/20 [00:12<01:15,  4.44s/it]

Professional role 1, 1/174:  20%|███▏            | 4/20 [00:13<00:51,  3.21s/it]

Professional role 1, 1/174:  30%|████▊           | 6/20 [00:14<00:22,  1.60s/it]

Professional role 1, 1/174:  40%|██████▍         | 8/20 [00:14<00:11,  1.03it/s]

Professional role 1, 1/174:  50%|███████▌       | 10/20 [00:14<00:06,  1.55it/s]

Professional role 1, 1/174:  60%|█████████      | 12/20 [00:14<00:03,  2.21it/s]

Professional role 1, 1/174:  65%|█████████▊     | 13/20 [00:14<00:02,  2.62it/s]

Professional role 1, 1/174:  70%|██████████▌    | 14/20 [00:14<00:01,  3.13it/s]

Professional role 1, 1/174:  75%|███████████▎   | 15/20 [00:15<00:01,  3.66it/s]

Professional rol

(43, 15)




Professional role 2, 2/174:   0%|                        | 0/20 [00:00<?, ?it/s]

Professional role 2, 2/174:  10%|█▌              | 2/20 [00:10<01:33,  5.21s/it]

Professional role 2, 2/174:  15%|██▍             | 3/20 [00:16<01:34,  5.54s/it]

Professional role 2, 2/174:  20%|███▏            | 4/20 [00:23<01:37,  6.11s/it]

Professional role 2, 2/174:  25%|████            | 5/20 [00:24<01:02,  4.17s/it]

Professional role 2, 2/174:  30%|████▊           | 6/20 [00:24<00:42,  3.06s/it]

Professional role 2, 2/174:  40%|██████▍         | 8/20 [00:25<00:19,  1.62s/it]

Professional role 2, 2/174:  45%|███████▏        | 9/20 [00:25<00:13,  1.23s/it]

Professional role 2, 2/174:  55%|████████▎      | 11/20 [00:25<00:06,  1.34it/s]

Professional role 2, 2/174:  60%|█████████      | 12/20 [00:25<00:05,  1.47it/s]

Professional role 2, 2/174:  70%|██████████▌    | 14/20 [00:25<00:02,  2.24it/s]

Professional role 2, 2/174:  75%|███████████▎   | 15/20 [00:26<00:01,  2.70it/s]

Professional r

(43, 15)




Professional role 3, 3/174:   0%|                        | 0/20 [00:00<?, ?it/s]

Professional role 3, 3/174:  10%|█▌              | 2/20 [00:06<00:58,  3.22s/it]

Professional role 3, 3/174:  15%|██▍             | 3/20 [00:14<01:25,  5.03s/it]

Professional role 3, 3/174:  20%|███▏            | 4/20 [00:20<01:26,  5.43s/it]

Professional role 3, 3/174:  25%|████            | 5/20 [00:26<01:27,  5.83s/it]

Professional role 3, 3/174:  30%|████▊           | 6/20 [00:32<01:20,  5.75s/it]

Professional role 3, 3/174:  35%|█████▌          | 7/20 [00:40<01:26,  6.64s/it]

Professional role 3, 3/174:  40%|██████▍         | 8/20 [00:48<01:24,  7.07s/it]

Professional role 3, 3/174:  45%|███████▏        | 9/20 [00:55<01:14,  6.80s/it]

Professional role 3, 3/174:  50%|███████▌       | 10/20 [00:57<00:54,  5.41s/it]

Professional role 3, 3/174:  55%|████████▎      | 11/20 [00:57<00:34,  3.79s/it]

Professional role 3, 3/174:  60%|█████████      | 12/20 [00:57<00:21,  2.67s/it]

Professional r

...

(14660, 15)




Professional role 173, 173/174:   0%|                    | 0/20 [00:00<?, ?it/s]

Professional role 173, 173/174:  10%|█▏          | 2/20 [00:05<00:50,  2.83s/it]

Professional role 173, 173/174:  15%|█▊          | 3/20 [00:12<01:13,  4.34s/it]

Professional role 173, 173/174:  20%|██▍         | 4/20 [00:18<01:20,  5.03s/it]

Professional role 173, 173/174:  25%|███         | 5/20 [00:24<01:22,  5.50s/it]

Professional role 173, 173/174:  30%|███▌        | 6/20 [00:25<00:55,  3.99s/it]

Professional role 173, 173/174:  35%|████▏       | 7/20 [00:25<00:35,  2.74s/it]

Professional role 173, 173/174:  40%|████▊       | 8/20 [00:25<00:23,  1.96s/it]

Professional role 173, 173/174:  45%|█████▍      | 9/20 [00:26<00:16,  1.53s/it]

Professional role 173, 173/174:  55%|██████     | 11/20 [00:26<00:07,  1.17it/s]

Professional role 173, 173/174:  65%|███████▏   | 13/20 [00:26<00:03,  1.81it/s]

Professional role 173, 173/174:  70%|███████▋   | 14/20 [00:27<00:03,  1.62it/s]

Professional r

(14666, 15)




Professional role 174, 174/174:   0%|                    | 0/20 [00:00<?, ?it/s]

Professional role 174, 174/174:  10%|█▏          | 2/20 [00:05<00:51,  2.86s/it]

Professional role 174, 174/174:  15%|█▊          | 3/20 [00:08<00:45,  2.70s/it]

Professional role 174, 174/174:  25%|███         | 5/20 [00:08<00:19,  1.28s/it]

Professional role 174, 174/174:  35%|████▏       | 7/20 [00:08<00:09,  1.32it/s]

Professional role 174, 174/174:  40%|████▊       | 8/20 [00:09<00:08,  1.34it/s]

Professional role 174, 174/174:  45%|█████▍      | 9/20 [00:09<00:06,  1.72it/s]

Professional role 174, 174/174:  55%|██████     | 11/20 [00:09<00:03,  2.65it/s]

Professional role 174, 174/174:  65%|███████▏   | 13/20 [00:09<00:01,  3.67it/s]

Professional role 174, 174/174:  70%|███████▋   | 14/20 [00:09<00:01,  4.20it/s]

Professional role 174, 174/174:  80%|████████▊  | 16/20 [00:10<00:00,  5.47it/s]

Professional role 174, 174/174:  90%|█████████▉ | 18/20 [00:10<00:00,  6.52it/s]

Processing are

(14702, 15)


In [225]:
with open('hh_logs.log', 'r') as file:
    logs = file.readlines()

print(logs[0].strip())

2024-05-09 17:56:53,931 - INFO - Successful concatenation. Shape of dataframe: (1, 15)


In [298]:
df_part

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,employment,workSchedule,workExperience,description,published_at,alternate_url,prof_codes
0,97239592,Event-менеджер (стажер),2634572,"[Обучение и развитие, Коммуникабельность, Even...",None,None,None,2,project,flexible,noExperience,Стажировка для event-менеджеров во «Вместе Заж...,2024-05-11 16:04:25,https://hh.ru/vacancy/97239592,1
1,98620090,Event Manager в EdTech,10610642,"[Организация мероприятий, Организаторские навы...",None,100000,RUR,2,full,remote,between1And3,Neogenda – адепты результативных подходов и пр...,2024-05-11 14:40:50,https://hh.ru/vacancy/98620090,1
2,97221828,Event-менеджер по организации молодежных мероп...,4464460,"[Управление проектами, Организаторские навыки,...",None,85000,RUR,2,full,remote,between3And6,Фонд «Надежная смена» с 2007 года реализует мо...,2024-05-11 13:15:58,https://hh.ru/vacancy/97221828,1
3,98213871,Комьюнити-менеджер,5676843,"[Работа в команде, Организаторские навыки, Гра...",55000,65000,RUR,2,full,shift,noExperience,"Команда SkateTown разыскивает друга, полного с...",2024-05-11 10:23:07,https://hh.ru/vacancy/98213871,1
4,98597185,Ассистент в отдел маркетинга и PR,10842666,"[Обучение и развитие, Планирование карьеры, Ор...",65000,75000,RUR,2,full,fullDay,noExperience,"Эта вакансия для тех, кто хочет плавно погрузи...",2024-05-11 10:12:28,https://hh.ru/vacancy/98597185,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14697,91549156,Начальник участка по специальным работам и ПНР...,1680355,[],None,None,None,2,full,flyInFlyOut,between3And6,Обязанности: Разработка и сопровождение прогр...,2024-05-02 11:01:05,https://hh.ru/vacancy/91549156,174
14698,98083829,Инженер АСУ,3796361,"[АСУ ТП, Автоматизация технологических процесс...",None,None,None,2,full,fullDay,between1And3,Требования Высшее техническое образование в о...,2024-04-27 16:04:11,https://hh.ru/vacancy/98083829,174
14699,95715187,Инженер-регулировщик,746659,"[MATLAB, Работа с чертежами, Работа с измерите...",70000,None,RUR,2,full,fullDay,between1And3,"Государственный научный центр АО Концерн ""ЦНИИ...",2024-04-27 15:49:11,https://hh.ru/vacancy/95715187,174
14700,93482970,Ведущий инженер (промышленное оборудование),8967,[],None,None,None,2,full,fullDay,moreThan6,Обязанности: Написание технических заданий на...,2024-04-27 12:03:14,https://hh.ru/vacancy/93482970,174


In [299]:
len(df_part['vacancy_id'].unique())

14701

In [300]:
df_new = df_part.dropna(subset=['name'])
df_new = df_new.drop_duplicates(subset=['vacancy_id'])

In [312]:
df_new.head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,employment,workSchedule,workExperience,description,published_at,alternate_url,prof_codes
0,97239592,Event-менеджер (стажер),2634572,"[Обучение и развитие, Коммуникабельность, Even...",None,None,None,2,project,flexible,noExperience,Стажировка для event-менеджеров во «Вместе Заж...,2024-05-11 16:04:25,https://hh.ru/vacancy/97239592,1
1,98620090,Event Manager в EdTech,10610642,"[Организация мероприятий, Организаторские навы...",None,100000,RUR,2,full,remote,between1And3,Neogenda – адепты результативных подходов и пр...,2024-05-11 14:40:50,https://hh.ru/vacancy/98620090,1
2,97221828,Event-менеджер по организации молодежных мероп...,4464460,"[Управление проектами, Организаторские навыки,...",None,85000,RUR,2,full,remote,between3And6,Фонд «Надежная смена» с 2007 года реализует мо...,2024-05-11 13:15:58,https://hh.ru/vacancy/97221828,1
3,98213871,Комьюнити-менеджер,5676843,"[Работа в команде, Организаторские навыки, Гра...",55000,65000,RUR,2,full,shift,noExperience,"Команда SkateTown разыскивает друга, полного с...",2024-05-11 10:23:07,https://hh.ru/vacancy/98213871,1
4,98597185,Ассистент в отдел маркетинга и PR,10842666,"[Обучение и развитие, Планирование карьеры, Ор...",65000,75000,RUR,2,full,fullDay,noExperience,"Эта вакансия для тех, кто хочет плавно погрузи...",2024-05-11 10:12:28,https://hh.ru/vacancy/98597185,1


In [306]:
df_new.to_csv(filename, index=False, mode='a')

In [307]:
new_data = pd.read_csv(filename)
new_data.shape

/var/folders/35/_fnsgly951b8wkst1vg_3lj40000gn/T/ipykernel_33529/2875217823.py:1: DtypeWarning: Columns (0,2,4,5,7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  new_data = pd.read_csv(filename)


(116835, 15)

In [313]:
new_data.shape

(116835, 15)

In [ ]:
# для апдейта можно period поменьше брать

In [219]:
# draft to check fields
response = requests.get(f"{url_vacancies}/{row['vacancy_id']}")

if response.status_code == 200:
    vac = response.json()
    print(vac['alternate_url'])

https://hh.ru/vacancy/92767727


Utils для сборки специальных датасетов

In [72]:
# сбор датасета для проверки гипотезы значимости prof_code для кластеризации
# Мoscow

filename = 'vacancy_hh_dataset_moscow.csv'
page_cnt = 25
item_per_page = 20
period = 365
area_code = 1
    
params = {
    'area': area_code,
    'currency_code': currency_code, 
    'per_page': item_per_page,
    'period': period,
    'order_by': order_by
}

df_msc = make_dataset(params, page_cnt)

df_msc_new = df_msc.dropna(subset=['name'])
df_msc_new = df_msc_new.drop_duplicates(subset=['vacancy_id'])
df_msc_new.to_csv(filename, index=False, mode='a')

In [42]:
# SPb

filename = 'vacancy_hh_dataset_spb.csv'
page_cnt = 25
item_per_page = 20
period = 365
area_code = 2
    
params = {
    'area': area_code,
    'currency_code': currency_code, 
    'per_page': item_per_page,
    'period': period,
    'order_by': order_by
}

df_spb = make_dataset(params, page_cnt)

df_spb_new = df_spb.dropna(subset=['name'])
df_spb_new = df_spb_new.drop_duplicates(subset=['vacancy_id'])
df_spb_new.to_csv(filename, index=False, mode='a')